In [ ]:
import pandas as pd
from dateutil import parser
import numpy as np
import re
from geopy.geocoders import Nominatim
import time
import math

# Question 1

# Function to parse dates
def parse_date(date_str):
    try:
        # Use dateutil parser with dayfirst=True for mixed formats
        date_time = parser.parse(date_str, dayfirst=True)
        return date_time.date()
    except Exception:
        return pd.NaT

# Apply parsing function
# df['clean_date'] = df['order_date'].apply(parse_date)

# Convert to standardized format 'YYYY-MM-DD'
# df['clean_date'] = pd.to_datetime(df['clean_date']).dt.strftime('%Y-%m-%d')

# If desired, replace NaT formatted strings with None or leave as-is
# df.loc[df['clean_date'] == 'NaT', 'clean_date'] = None




# Question 2

def clean_price(value):
    if isinstance(value, (int, float)):  # Already numeric
        return value
    if isinstance(value, str):
        # Remove currency symbols and commas, keep digits and dot
        cleaned = re.sub(r'[^\d.]', '', value)
        try:
            return float(cleaned) if '.' in cleaned else int(cleaned)
        except ValueError:
            return None
    return None # For other types (e.g., None)

# Example usage
# df['original_price_inr_cleaned'] = df['original_price_inr'].apply(clean_price)




# Question 3

def round_up_scale(value):
    """Round value up to nearest 5 or 10, whichever is appropriate."""
    if value <= 5:
        return 5
    elif value <= 10:
        return 10
    else:
        # For values above 10, round up to nearest multiple of 5
        return int(math.ceil(value / 5.0)) * 5

def standardize_rating(rating):
    if rating is None:
        return np.nan

    rating_str = str(rating).strip().lower()

    # Fraction parsing
    if '/' in rating_str:
        try:
            numerator, denominator = rating_str.split('/')
            numerator = float(numerator)
            denominator = float(denominator)
            if denominator == 0:
                return np.nan
            normalized = (numerator / denominator) * 5
            if 1.0 <= normalized <= 5.0:
                return round(normalized, 2)
            else:
                return np.nan
        except:
            return np.nan

    # Extract numeric value
    match = re.search(r'(\d+(\.\d+)?)', rating_str)
    if match:
        try:
            value = float(match.group(1))

            scale = round_up_scale(value)
            normalized = (value / scale) * 5

            # Clamp between 1 and 5
            normalized = max(1.0, min(normalized, 5.0))
            return round(normalized, 2)

        except:
            return np.nan

    return np.nan

# df["customer_rating_cleaned"] = df["customer_rating"].apply(standardize_rating)






# Question 4

# Sample city names (including variants and misspellings)
# cities = ['Bangalore', 'bengaluru', 'Mumbai', 'Bombay', 'delhi', 'New Delhi', 'mumbay', 'banaglore', 'DELHI']

# df = pd.DataFrame({'customer_city': cities})

# Initialize Nominatim geocoder with a user_agent
geolocator = Nominatim(user_agent="osm_city_standardizer")

def geocode_city_osm(city):
    try:
        # Limit search to India for better accuracy
        location = geolocator.geocode(f"{city}, India", language='en', exactly_one=True, addressdetails=True)
        if location and 'address' in location.raw:
            address = location.raw['address']
            # Extract city or town from the address details
            city_name = address.get('city') or address.get('town') or address.get('village')
            
            # Sometimes the city might be under 'state_district' or 'county' if city not found
            if not city_name:
                city_name = address.get('state_district') or address.get('county')
            
            if city_name:
                return city_name
            else:
                # Fallback: use first part of display_name
                return location.address.split(',')[0]
        else:
            return None
    except Exception as e:
        print(f"Error geocoding '{city}': {e}")
        return None

# Step 1: Get unique city names
unique_cities = df['customer_city'].unique()
# print(f"the len of unique cities: {len(unique_cities)}")

# Step 2: Build a mapping dictionary
city_mapping = {}

for city in unique_cities:
    print(f"input city is: {city}")
    standardized_city = geocode_city_osm(city)
    print(f"output city is: {standardized_city}")
    city_mapping[city] = standardized_city
    time.sleep(1)  # Pause to respect Nominatim's usage policy (1 sec per request)

# Step 3: Map standardized values back to the original DataFrame
# df['customer_city_standardized'] = df['customer_city'].map(city_mapping)

# to handle
"""
input city is: mumba
output city is: Navi Mumbai
input city is: chenai
output city is: None
input city is: Delhi NCR
output city is: Gurugram
"""

## method 2

import pandas as pd
from rapidfuzz import process, fuzz

# Load your data
df = pd.read_csv("your_file.csv")  # Replace with your file name
input_cities = df["input_city"].astype(str)

# Assume your true city names list is available
true_cities = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix"]  # Replace with your actual list

# Define a function to find the closest match
def get_best_match(city, choices, threshold=80):
    match, score, _ = process.extractOne(city, choices, scorer=fuzz.WRatio)
    return match if score >= threshold else city  # fallback to original if no good match

# Apply the function to the dataframe
df["clean_city"] = input_cities.apply(lambda x: get_best_match(x, true_cities))

# Save the corrected file
df.to_csv("corrected_cities.csv", index=False)




# Question 5

# Define a function to normalize boolean-like values






# Question 6

df['Category_Standardized'] = 'Electronics'





# Question 7

def clean_day(value):
    value = value.strip().lower()

    # Special case: 'same day'
    if value == "same day":
        return 0

    # Check for two numbers separated by a dash (e.g., "1-2 days")
    range_match = re.findall(r"\d+(?:\.\d+)?", value)
    if '-' in value:
        if len(range_match) == 2:
            try:
                return max(float(range_match[0]), float(range_match[1]))
            except:
                return np.nan
        else:
            return np.nan

    # Check if it's a single valid number (possibly with 'days' word)
    if len(range_match) == 1:
        try:
            num = float(range_match[0])
            if num < 0:
                return np.nan
            return num
        except:
            return np.nan

    # If none of the above matched, it's invalid
    return np.nan






# Question 8

# Step 1: Find groups with potential duplicates
grouped = df.groupby(['customer_id', 'product_id', 'order_date', 'final_amount_inr'])

def analyze_group(group):
    unique_tx_ids = group['transaction_id'].nunique()
    total_records = len(group)
    
    if unique_tx_ids == total_records:
        # All transactions have unique IDs -> likely genuine bulk order
        return 'genuine_bulk_order'
    elif unique_tx_ids < total_records:
        # Some transaction_ids repeat -> data error duplicates
        return 'data_error_duplicates'
    else:
        return 'ambiguous'

# Apply the function to each group and create a Series with the result repeated for each row in the group
duplicate_types = grouped.apply(analyze_group).reset_index(name='duplicate_type')

# Map the group keys back to the original dataframe by merging or joining
df = df.merge(duplicate_types.reset_index(), on=['customer_id', 'product_id', 'order_date', 'final_amount_inr'], how='left')







# Question 9

def correct_prices(group):
    median_price = group['original_price_inr_cleaned'].median()
    threshold = 3 * median_price
    
    # Create new column 'corrected_price' starting as copy of original 'price'
    group['corrected_price'] = group['original_price_inr_cleaned']
    
    outliers = group['original_price_inr_cleaned'] > threshold
    correction_candidates = (group['original_price_inr_cleaned'] / 100) < threshold
    
    # Only update 'corrected_price' column; original 'price' unchanged
    group.loc[outliers & correction_candidates, 'corrected_price'] = group.loc[outliers & correction_candidates, 'original_price_inr_cleaned'] / 100
    
    return group

# df = df.groupby('product_id').apply(correct_prices).reset_index(drop=True)

# Apply function per product
df = df.groupby('product_id').apply(correct_prices).reset_index(drop=True)






# Question 10

def standardize_payment_method(payment_method: str) -> str:
    payment_method = str(payment_method).lower()  # Convert to string and lowercase

    payment_map = {
        'UPI': ['upi', 'phonepe', 'googlepay'],
        'Credit Card': ['credit card', 'credit_card', 'cc'],
        'Debit Card': ['debit card', 'debit_card', 'dc'],
        'Cash on Delivery': ['cash on delivery', 'cod', 'c.o.d'],
        # Add more mappings if needed
    }

    for category, keywords in payment_map.items():
        if any(keyword in payment_method for keyword in keywords):
            return category
    return 'Other'

# Apply the function to the payment_method column
df['standard_payment_method'] = df['payment_method'].apply(standardize_payment_method)

In [ ]:
import pandas as pd
from dateutil import parser
import numpy as np
import re
from geopy.geocoders import Nominatim
import time
import math


class DataCleaning():
    def __init__(self):
        self.city_names_corrections = {
            "chenai": "chennai",
            "delhi ncr": "delhi",
            "navi mumbai": "mumbai",
        }
        self.payment_map = {
            'UPI': ['upi', 'phonepe', 'googlepay'],
            'Credit Card': ['credit card', 'credit_card', 'cc'],
            'Debit Card': ['debit card', 'debit_card', 'dc'],
            'Cash on Delivery': ['cash on delivery', 'cod', 'c.o.d'],
            "Net Banking": ['net banking', 'net_banking', 'nb'],
            "Wallet": ['wallet'],
            "Buy Now Pay Later": ['buy now pay later', 'bnpl'],
            # Add more mappings if needed
        }
        self.max_delivery_days = 15  # Set a maximum threshold for delivery days

    # Question 1: Date Cleaning
    def date_cleaning(self, date_str):
        try:
            # Use dateutil parser with dayfirst=True for mixed formats
            date_time = parser.parse(date_str, dayfirst=True)
            return date_time.date()
        except Exception:
            return None
    
    # Question 2: Price Cleaning
    def price_cleaning(self, value):
        if isinstance(value, (int, float)):  # Already numeric
            return value
        if isinstance(value, str):
            # Remove currency symbols and commas, keep digits and dot
            cleaned = re.sub(r'[^\d.]', '', value)
            try:
                return float(cleaned) if '.' in cleaned else int(cleaned)
            except ValueError:
                return np.nan
        return np.nan # For other types (e.g., None)
    
    # Question 3: Rating Standardization
    def round_up_scale(self, value):
        """Round value up to nearest 5 or 10, whichever is appropriate."""
        if value <= 5:
            return 5
        elif value <= 10:
            return 10
        else:
            # For values above 10, round up to nearest multiple of 5
            return int(math.ceil(value / 5.0)) * 5
        
    def standardize_rating(self, rating):
        if rating is None:
            return np.nan

        rating_str = str(rating).strip().lower()

        # Fraction parsing
        if '/' in rating_str:
            try:
                numerator, denominator = rating_str.split('/')
                numerator = float(numerator)
                denominator = float(denominator)
                if denominator == 0:
                    return np.nan
                normalized = (numerator / denominator) * 5
                if 1.0 <= normalized <= 5.0:
                    return round(normalized, 2)
                else:
                    return np.nan
            except:
                return np.nan

        # Extract numeric value
        match = re.search(r'(\d+(\.\d+)?)', rating_str)
        if match:
            try:
                value = float(match.group(1))

                scale = self.round_up_scale(value)
                normalized = (value / scale) * 5

                # Clamp between 1 and 5
                normalized = max(1.0, min(normalized, 5.0))
                return round(normalized, 2)

            except:
                return np.nan
        return np.nan
    
    # Question 4: City Name Standardization
    def geocode_city_osm(self, city, geolocator):
        try:
            # Limit search to India for better accuracy
            location = geolocator.geocode(f"{city}, India", language='en', exactly_one=True, addressdetails=True)
            if location and 'address' in location.raw:
                address = location.raw['address']
                # Extract city or town from the address details
                city_name = address.get('city') or address.get('town') or address.get('village')
                
                # Sometimes the city might be under 'state_district' or 'county' if city not found
                if not city_name:
                    city_name = address.get('state_district') or address.get('county')
                
                if city_name:
                    return city_name
                else:
                    # Fallback: use first part of display_name
                    return location.address.split(',')[0]
            else:
                return None
        except Exception as e:
            print(f"Error geocoding '{city}': {e}")
            return None
    
    def city_standardization(self, unique_cities):
        geolocator = Nominatim(user_agent="osm_city_standardizer")

        # Step 1: Get unique city names
        for i in range(len(unique_cities)):
            if unique_cities[i].lower() in self.city_names_corrections:
                unique_cities[i] = self.city_names_corrections[unique_cities[i].lower()]

        # Step 2: Build a mapping dictionary
        city_mapping = {}

        for city in unique_cities:
            print(f"input city is: {city}")
            standardized_city = self.geocode_city_osm(city=city, geolocator=geolocator)
            print(f"output city is: {standardized_city}")
            city_mapping[city] = standardized_city
            time.sleep(1)  # Pause to respect Nominatim's usage policy (1 sec per request)
        
        for key, value in city_mapping.items():
            city_mapping[key] = self.city_names_corrections.get(value.lower(), value).title()
            print(f"after correction city is: {key, city_mapping[key]}")
        return city_mapping
    
    # Question 5: Boolean Normalization
    def normalize_boolean(self, val):
        if pd.isna(val):
            return False  # or pd.NA if you want to keep missing values
        if isinstance(val, str):
            val = val.strip().lower()
            if val in ['yes', 'y', 'true', '1']:
                return True
            elif val in ['no', 'n', 'false', '0']:
                return False
            else:
                return pd.NA  # unrecognized string
        return bool(val)
    
    # Question 6: Category Standardization
    def standardize_category(self, categories):
        output = {}
        for i in categories:
            output[i] = "Electronics" if "electronic" in i.lower() else i
        return output
    
    # Question 7: Delivery Days Cleaning    
    def clean_delivery_day(self, value, max_valid_days=None):
        """
        Cleans delivery day input and returns a numeric value representing delivery days.
        Returns np.nan for invalid or unrealistic values.
        """
        if not max_valid_days:
            max_valid_days = self.max_delivery_days
        if not isinstance(value, str):
            return np.nan

        value = value.strip().lower()

        # Special case: 'same day'
        if value == "same day":
            return 0

        # Extract all numeric parts (integers or decimals)
        range_match = re.findall(r"\d+(?:\.\d+)?", value)

        if '-' in value or 'to' in value:
            # Handle range like "1-2 days" or "1 to 3 business days"
            if len(range_match) == 2:
                try:
                    high = max(float(range_match[0]), float(range_match[1]))
                    return high if high <= max_valid_days else np.nan
                except:
                    return np.nan
            else:
                return np.nan

        # Single number case (e.g., '2 days', '3 business days')
        if len(range_match) == 1:
            try:
                num = float(range_match[0])
                if num < 0 or num > max_valid_days:
                    return np.nan
                return num
            except:
                return np.nan

        # Unrecognized format
        return np.nan
    
    # Question 8: Duplicate Transaction Identification
    def analyze_group(self, group):
        unique_tx_ids = group['transaction_id'].nunique()
        total_records = len(group)
        
        # All unique transaction IDs
        if unique_tx_ids == total_records:
            return 'genuine_bulk_order'
        
        # Repeated transaction IDs with identical other features
        tx_id_counts = group['transaction_id'].value_counts()
        repeated_tx_ids = tx_id_counts[tx_id_counts > 1].index

        for tx_id in repeated_tx_ids:
            repeated_tx_group = group[group['transaction_id'] == tx_id]
            # Check if all rows for this transaction ID are exactly the same
            if repeated_tx_group.drop(columns='transaction_id').duplicated().all():
                return 'data_error_duplicates'
        
        # Could be mix of genuine and error
        return 'ambiguous'
    
    # Question 9: Price Correction
    def mad_based_threshold(self, group, column_name, threshold_multiplier=3):
        median_price = group[column_name].median()
        mad = (group[column_name] - median_price).abs().median()
        threshold = median_price + threshold_multiplier * mad
        return threshold


    def correct_prices(self, group, threshold_multiplier=3):
        median_price = group['clean_original_price_inr'].median()
        # threshold = threshold_multiplier * median_price
        threshold = self.mad_based_threshold(group, column_name='clean_original_price_inr', threshold_multiplier=threshold_multiplier)
        
        # Start corrected_price as a copy to avoid SettingWithCopyWarning
        group = group.copy()
        group['corrected_price'] = group['clean_original_price_inr']
        
        # Identify outliers beyond threshold
        outliers = group['clean_original_price_inr'] > threshold
        
        # Check if dividing by 100 brings price below threshold
        correction_candidates = (group['clean_original_price_inr'] / 100) < threshold
        
        # Apply correction where both conditions hold
        condition = outliers & correction_candidates
        group.loc[condition, 'corrected_price'] = group.loc[condition, 'clean_original_price_inr'] / 100

        # Optional: print or log number of corrections made
        num_corrected = condition.sum()
        # if num_corrected > 0:
        #     print(f"Corrected {num_corrected} prices in product_id {group.name}")
        
        return group
    
    # Question 10: Payment Method Standardization
    def standardize_payment_method(self, payment_method: str) -> str:
        payment_method = str(payment_method).lower()  # Convert to string and lowercase

        for category, keywords in self.payment_map.items():
            if any(keyword in payment_method for keyword in keywords):
                return category
        return 'Other'

# [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
years = [2025]
for i in years:
    input_path = fr"C:\Users\haris\OneDrive\Desktop\Guvi\Projects\Amazon_Sales_Analysis\dataset\amazon_india_{i}.csv"
    output_path = fr"C:\Users\haris\OneDrive\Desktop\Guvi\Projects\Amazon_Sales_Analysis\dataset\amazon_india_{i}_cleaned.csv"
    df = pd.read_csv(input_path)
    data_cleaner = DataCleaning()

    # Question 1
    df["clean_order_date"] = df["order_date"].apply(data_cleaner.date_cleaning)

    # Question 2
    columns = ["original_price_inr", "discount_percent", "final_amount_inr", "delivery_charges"]
    df[["clean_original_price_inr", "clean_discount_percent", "clean_final_amount_inr", "clean_delivery_charges"]] = df[columns].map(data_cleaner.price_cleaning)

    # Question 3
    columns = ['customer_rating', 'product_rating']
    df[["cleaned_customer_rating", "cleaned_product_rating"]] = df[columns].map(data_cleaner.standardize_rating)

    # Question 4
    cleaned_citys = data_cleaner.city_standardization(df['customer_city'].astype(str).str.lower().unique())
    df["cleaned_customer_city"] = df["customer_city"].astype(str).str.lower().map(cleaned_citys)

    # Question 5
    columns = ['is_prime_member', 'is_prime_eligible', 'is_festival_sale']
    df[['cleaned_is_prime_member', 'cleaned_is_prime_eligible', 'cleaned_is_festival_sale']] = df[columns].map(data_cleaner.normalize_boolean)

    # Question 6
    category_map = data_cleaner.standardize_category(df["category"].astype(str).unique())
    df["cleaned_category"] = df["category"].map(category_map)

    # Question 7
    df["cleaned_delivery_days"] = df["delivery_days"].apply(data_cleaner.clean_delivery_day)

    # Question 8
    grouped = df.groupby(['customer_id', 'product_id', 'order_date', 'final_amount_inr'])
    # Apply the function to each group and create a Series with the result repeated for each row in the group
    duplicate_types = grouped.apply(data_cleaner.analyze_group, include_groups=False).reset_index(name='duplicate_type')

    # Map the group keys back to the original dataframe by merging or joining
    df = df.merge(duplicate_types.reset_index(), on=['customer_id', 'product_id', 'order_date', 'final_amount_inr'], how='left')
    if 'index' in df.columns:
        df.drop(columns='index', inplace=True)

    # Question 9
    df = df.groupby('product_id').apply(data_cleaner.correct_prices).reset_index(drop=True)

    # Question 10
    df['standard_payment_method'] = df['payment_method'].apply(data_cleaner.standardize_payment_method)

    # save the output to a csv file
    df.to_csv(output_path, index=False)
    print(f"cleaned data saved to {output_path}")
    print("\n\n")

In [205]:
df.columns

Index(['transaction_id', 'order_date', 'customer_id', 'product_id',
       'product_name', 'category', 'subcategory', 'brand',
       'original_price_inr', 'discount_percent', 'discounted_price_inr',
       'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr',
       'customer_city', 'customer_state', 'customer_tier',
       'customer_spending_tier', 'customer_age_group', 'payment_method',
       'delivery_days', 'delivery_type', 'is_prime_member', 'is_festival_sale',
       'festival_name', 'customer_rating', 'return_status', 'order_month',
       'order_year', 'order_quarter', 'product_weight_kg', 'is_prime_eligible',
       'product_rating'],
      dtype='object')

In [ ]:
df.to_csv(r"C:\Users\haris\OneDrive\Desktop\Guvi\Projects\Amazon_Sales_Analysis\dataset\amazon_india_2017_cleaned.csv", index=False)

In [ ]:
df[["order_date", "clean_order_date"]]

In [ ]:
df[["original_price_inr", "clean_original_price_inr", "discount_percent", "clean_discount_percent", "final_amount_inr", "clean_final_amount_inr", "delivery_charges", "clean_delivery_charges"]]

In [ ]:
df[["product_rating", "cleaned_product_rating", "customer_rating", "cleaned_customer_rating"]]

In [ ]:
df[["is_prime_member", "cleaned_is_prime_member", "is_prime_eligible", "cleaned_is_prime_eligible", "is_festival_sale", "cleaned_is_festival_sale"]]

In [154]:
pd.unique(df[["is_prime_member", "cleaned_is_prime_member", "is_prime_eligible", "cleaned_is_prime_eligible", "is_festival_sale", "cleaned_is_festival_sale"]].values.ravel())

array(['False', False, 'True', True, 'FALSE', '0', 'No', 'Yes', 'TRUE',
       '1'], dtype=object)

In [155]:
pd.unique(df[["cleaned_is_prime_member", "cleaned_is_prime_eligible", "cleaned_is_festival_sale"]].values.ravel())

array([False,  True])

In [201]:
df["category"].unique()

array(['Electronics', 'ELECTRONICS', 'Electronicss',
       'Electronics & Accessories', 'Electronic'], dtype=object)

In [129]:
df[["category", "cleaned_category"]]

,category,cleaned_category
0,Electronics,Electronics
1,Electronics,Electronics
2,Electronics,Electronics
3,Electronics,Electronics
4,Electronics,Electronics
...,...,...
55270,Electronics,Electronics
55271,Electronics,Electronics
55272,Electronics,Electronics
55273,Electronics,Electronics


In [136]:
df["duplicate_type"].unique()

array(['genuine_bulk_order'], dtype=object)

In [200]:
df["payment_method"].unique()

array(['UPI', 'Credit Card', 'BNPL', 'COD', 'Debit Card', 'Net Banking',
       'Wallet'], dtype=object)

In [206]:
df = pd.read_csv(r"C:\Users\haris\OneDrive\Desktop\Guvi\Projects\Amazon_Sales_Analysis\dataset\cleaned_dataset\amazon_india_2025_cleaned.csv")

In [207]:
df.columns

Index(['transaction_id', 'order_date', 'customer_id', 'product_id',
       'product_name', 'category', 'subcategory', 'brand',
       'original_price_inr', 'discount_percent', 'discounted_price_inr',
       'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr',
       'customer_city', 'customer_state', 'customer_tier',
       'customer_spending_tier', 'customer_age_group', 'payment_method',
       'delivery_days', 'delivery_type', 'is_prime_member', 'is_festival_sale',
       'festival_name', 'customer_rating', 'return_status', 'order_month',
       'order_year', 'order_quarter', 'product_weight_kg', 'is_prime_eligible',
       'product_rating', 'clean_order_date', 'clean_original_price_inr',
       'clean_discount_percent', 'clean_final_amount_inr',
       'clean_delivery_charges', 'cleaned_customer_rating',
       'cleaned_product_rating', 'cleaned_customer_city',
       'cleaned_is_prime_member', 'cleaned_is_prime_eligible',
       'cleaned_is_festival_sale', 'cleaned_c